In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key) 

sk-proj-LHPdIFYw1wcQeVTI635JT3BlbkFJJRmD0xaCpJccu1A9F7A7


In [3]:
data = pd.read_csv('scenario.csv', encoding='utf-8')
data.head()

,인덱스,유형,QA
0,1,생물 보호,시간: 10:00\n상황: 어부들이 불법으로 산호를 채집하고 있다.\n절차: 1. ...
1,2,생물 보호,시간: 15:00\n상황: 관광객들이 불법으로 바다 거북 알을 채집하고 있다.\n절...
2,3,생물 보호,시간: 11:00\n상황: 다이버들이 불법으로 해삼을 채집하고 있다\n절차: 1. ...
3,4,생물 보호,시간: 16:00\n상황: 어린이가 해수욕장에서 해양 생물을 학대하고 있다.\n절차...
4,5,생물 보호,시간: 13:00\n상황: 어부들이 보호 구역에서 불법으로 물고기를 잡고 있다.\n...


In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
database = Chroma(persist_directory="./database", embedding_function = embeddings )

In [5]:
# 각 행의 데이터를 Document 객체로 변환 
documents = [Document(page_content=text) for text in data['QA'].tolist()]

# 데이터프레임에서 문서 추가
database.add_documents(documents)

['35a7ea37-2d1b-4839-9f94-56b107e8c780',
 '152cedca-e4e2-42cb-9510-1a734af0d970',
 'e09d632f-6362-46c4-b39f-33ba122053ab',
 '4cf32401-77a4-4998-8e58-5dea4a91495f',
 '5d84eb83-395f-4114-8049-cf7d884642e4',
 'db3955be-c981-4c61-aa56-12e35186b5b8',
 '8661df10-dd06-4d64-b982-7abac7924580',
 '6596878a-a1f6-406b-97e8-f744bef011c9',
 '369d0192-4740-472e-b46b-a10b6f8c7975',
 '97119bf0-ffa3-407a-86c3-175aafa26e67',
 '261e2aca-11dc-4242-96c1-122192c410a3',
 '87b57227-5770-489f-ace1-787a700ddf87',
 '9322bc3d-54dd-4a9a-8a48-0a1453218a43',
 'be97aa88-6a26-414e-849c-d3efcfea90e9',
 'ea55a6df-3ca8-4370-ab84-9a70992b89fa',
 '71330897-3338-43e0-95a1-d4fa9504d625',
 '2b83d93d-4338-49a0-a20a-3d810762a32a',
 '7eabf9fe-4de5-42ff-8eda-9bd3c8fde917',
 'e7dcd859-4cc1-4481-9247-1162dc1d0c2b',
 '85cbbaf8-6932-4a75-bfc7-6f3f26dd3238',
 'bdc48d72-2e76-49db-b224-3193d93cae36',
 '97e2a65c-5ad3-4dcf-b4d4-0540eb98f8a4',
 '2d796762-8607-4350-a3f6-96eb9f02bdbc',
 '24aefc8d-e729-4928-838f-6e434ba35c47',
 '5877d5f4-323f-

In [6]:
# 입력된 데이터 조회
database.get()

{'ids': ['007fbcf2-4650-4786-b815-6330fc2063f0',
  '06d6d91e-841a-437f-866e-c1065e374bea',
  '08718ea4-9409-453b-a898-6d54969bd515',
  '093c741a-e7a2-4bd7-a211-472b67ff1eba',
  '0b5d1dab-bfe0-4fa7-9414-035527a2e56e',
  '13e6b798-8992-49d9-959c-0c5ad1d2d8dc',
  '142a30aa-cc68-4175-8f2d-a44eb2b3dc9c',
  '14c06716-ec45-46d9-914a-c116aa071d8d',
  '151e1825-5396-4327-93f5-dfecbb724695',
  '152cedca-e4e2-42cb-9510-1a734af0d970',
  '15357372-2466-429f-afaf-e0cb8cf58bf6',
  '16002845-0360-4021-bc4a-142570dd3a4c',
  '1722480d-0af3-473b-8437-80f623b467a2',
  '18d8491f-c2d7-4e9d-97e6-ce04c5f27ff7',
  '1ad00d16-2360-458a-adcf-bfcc8af46be0',
  '1ba64bfe-f239-4210-890c-ef0d9a54fb26',
  '1fada359-8d31-4f1d-9f9a-3ce3c18f2568',
  '1fda902c-e13e-4ce0-8d61-4e3cf31f1371',
  '20253161-acc3-491a-b5a2-5e7f0ee7aa5f',
  '20e0a856-0640-4e23-bfc2-48cbf58ae7cf',
  '23ba6901-ce1e-465e-a318-47f69ed03522',
  '24aefc8d-e729-4928-838f-6e434ba35c47',
  '25d0f827-dad6-44ac-baf5-36fa0cdca900',
  '261e2aca-11dc-4242-96c1-

In [7]:
chat = ChatOpenAI(model="gpt-3.5-turbo")

k=20
retriever = database.as_retriever(search_kwargs={"k": k})

# 대화 메모리 생성
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer", 
                                  return_messages=True)

# ConversationalRetrievalQA 체인 생성
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory, 
                                           return_source_documents=True,  output_key="answer")

In [11]:
query = '유형: 불법, 상황: 해안가에서 무단으로 모래를 채취하는 무리가 발견됨.'
result = qa(query)

print(result["answer"])

해안가에서 무단으로 모래를 채취하는 무리를 발견했을 때, 다음 단계를 따라 조치를 취할 수 있습니다:
1. 신고를 접수합니다:
   - 신고자: 인근 주민
   - 신고 대상: 해양 경찰
   - 신고 내용: 불법 모래 채취
   - 시간 소요: 2분

2. 초기 대응:
   - 출동 인원: 해양 경찰 4명
   - 준비 작업: 채취 단속 장비 준비, 출동 경로 설정
   - 시간 소요: 3분

3. 현장 도착:
   - 도착 시간: 10분 이내
   - 출동 경로: 주민 위치에서 현장까지 직행

4. 단속 작업:
   - 단속 인원: 해양 경찰 4명
   - 작업 내용: 채취 장비 압수, 관련자 체포
   - 시간 소요: 10분

5. 후속 조치:
   - 조치 인원: 해양 경찰 2명
   - 조치 내용: 모래 채취 관련자 조사 및 법적 조치
   - 시간 소요: 5분

목표는 총 25분 이내에 불법 모래 채취를 중지하고 관련자를 체포하는 것입니다.
